In [1]:
## Segmentación del mercado de adolecentes en PySpark
##   En este tutorial se aplica el algoritmo K-means para clasificar un grupo de adolecentes con base en sus intéreses, con el fin de diseñar estrategias publicitarias 
##   y servicios encaminados a cada grupo de interés usando PySpark. Este tutorial se enfoca en la programación de PySpark y no en el análisis del problema

In [2]:
## Definición del problema
##   Un vendedor desea enviar publicidad electrónica a una población de adolecentes y adultos jóvenes con el fin de maximizar sus ventas. Para ello, desea poder clasificar 
##   a sus clientes potenciales por grupos de interés de acuerdo con sus intereses y consecuentemente enviar publicidad específica a cada uno de ellos.
##   En este problema se desea determina que grupos de interés existen en una población de clientes a partir de los mensajes enviados por un servicio de redes sociales. 
##   La información disponible consiste en 30000 observaciones de 40 variables que podrían caracterizar los intereses de la población analizada. Estas variables corresponden 
##   a palabras que pueden asociarse a un interés de la poblaión analizada. Cada variable mide la frecuencia con que una determinada palabra aparece en los mensajes de texto; 
##   adicionalmente, dentro de estas variables se incluye información como el sexo, la edad y la cantidad de contactos de la persona.

In [3]:
## Solución

In [4]:
## Preparación

In [5]:
##
## Se cargan librerías auxiliares que
## pueden ser de utilidad en el desarrollo
## del código
##
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
##
## Carga de las librerías de Spark
##
import findspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

findspark.init()

APP_NAME = "spark-kmeans-app"

conf = SparkConf().setAppName(APP_NAME)
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

In [7]:
## Carga de datos

In [8]:
## El archivo con los datos se encuentra en la carpeta actual de trabajo en la máquina local.

In [9]:
!wget https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/snsdata.csv

--2020-01-25 15:38:03--  https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/snsdata.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2631136 (2.5M) [text/plain]
Saving to: 'snsdata.csv'

snsdata.csv         100%[===================>]   2.51M   145KB/s    in 61s     

2020-01-25 15:39:42 (42.2 KB/s) - 'snsdata.csv' saved [2631136/2631136]



In [10]:
##
## Contenido del archivo
##
!head snsdata.csv

gradyear,gender,age,friends,basketball,football,soccer,softball,volleyball,swimming,cheerleading,baseball,tennis,sports,cute,sex,sexy,hot,kissed,dance,band,marching,music,rock,god,church,jesus,bible,hair,dress,blonde,mall,shopping,clothes,hollister,abercrombie,die,death,drunk,drugs
2006,M,18.982,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2006,F,18.801,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,2,1,0,0,0,6,4,0,1,0,0,0,0,0,0,0,0
2006,M,18.335,69,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2006,F,18.875,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2006,NA,18.995,10,0,0,0,0,0,0,0,0,0,0,0,1,0,0,5,1,1,0,3,0,1,0,0,0,1,0,0,0,2,0,0,0,0,0,1,1
2006,F,,142,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,2,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0
2006,F,18.93,72,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,2,0,0,2,0,0,0,0,0
2006,M,18.322,17,0,0,0,1,0,0,0,0,0,0,0,2,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2006,F,19.055

In [11]:
##
## Mueve el archivo de datos al hdfs
##
!hdfs dfs -copyFromLocal snsdata.csv /tmp/snsdata.csv

In [12]:
##
## Se carga el archivo en PySpark
##
spark_df = spark.read.load("/tmp/snsdata.csv",
                           format="csv",
                           sep=",",
                           inferSchema="true",
                           header="true")

##
## Número de registros cargados
##
spark_df.count()

30000

In [13]:
##
## Tipos de datos de los campos del DataFrame
##
spark_df.printSchema()

root
 |-- gradyear: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- friends: integer (nullable = true)
 |-- basketball: integer (nullable = true)
 |-- football: integer (nullable = true)
 |-- soccer: integer (nullable = true)
 |-- softball: integer (nullable = true)
 |-- volleyball: integer (nullable = true)
 |-- swimming: integer (nullable = true)
 |-- cheerleading: integer (nullable = true)
 |-- baseball: integer (nullable = true)
 |-- tennis: integer (nullable = true)
 |-- sports: integer (nullable = true)
 |-- cute: integer (nullable = true)
 |-- sex: integer (nullable = true)
 |-- sexy: integer (nullable = true)
 |-- hot: integer (nullable = true)
 |-- kissed: integer (nullable = true)
 |-- dance: integer (nullable = true)
 |-- band: integer (nullable = true)
 |-- marching: integer (nullable = true)
 |-- music: integer (nullable = true)
 |-- rock: integer (nullable = true)
 |-- god: integer (nullable = true)
 |-- church: inte

In [14]:
## Análisis exploratorio
##   A continuación se ejemplifican algunos cómputos típicos para el análisis exploratorio.

In [15]:
##
## Conteo por género
##
spark_df.groupBy('gender').count().toPandas()

,gender,count
0,F,22054
1,NA,2724
2,M,5222


In [16]:
##
## Se analizan los rangos de las variables
## para determinar si hay datos por fuera de sus
## rangos válidos. La variable `edad` contiene
## datos por fuera de la población de interés.
##
## La muestra contiene un rango de edades
## por fuera de la población de interés
##
spark_df.select('age').describe().toPandas()

,summary,age
0,count,24914
1,mean,17.993949546439772
2,stddev,7.858054477853863
3,min,3.086
4,max,106.927


In [17]:
##
## Se imprimen algunos registros para ver los
## valores del campo edad
##
spark_df.select('age').show()

+------+
|   age|
+------+
|18.982|
|18.801|
|18.335|
|18.875|
|18.995|
|  null|
| 18.93|
|18.322|
|19.055|
|18.708|
|18.543|
|19.463|
|18.097|
|  null|
|18.398|
|  null|
|  null|
|18.987|
|17.158|
|18.497|
+------+
only showing top 20 rows



In [18]:
##
## Cantidad de nulos en la columna age
##
spark_df.filter(spark_df['age'].isNull()).count()

5086

In [29]:
##
## Se seleccionan las personas entre 13 y 20 años y
## se descartan las demás observaciones
##
from pyspark.sql.functions import lit
from pyspark.sql.functions import when
from pyspark.sql.types import DoubleType

##
## Se agrega una columna con las edades entre 13 y 19,
## reemplazando por null los valores por fuera de este
## rango
##
spark_df = spark_df.withColumn(
    'age1319',
    when((spark_df['age'] >= 13) & (spark_df['age'] < 20),
         spark_df['age']
        ).otherwise(lit(None).cast(DoubleType())))

##
## Se verifican los valores en la nueva columna
##
spark_df.select('age1319').describe().toPandas()

,summary,age1319
0,count,24477
1,mean,17.25242893328433
2,stddev,1.1574649278955391
3,min,13.027
4,max,19.995


In [30]:
##
## Se calcula la edad promedio por año
## de graduación para la muestra en el
## rango de edades considerado
##
age_df = spark_df.groupby("gradyear").mean().select(['gradyear', 'avg(age1319)']).orderBy('gradyear')
age_df.show()

+--------+------------------+
|gradyear|      avg(age1319)|
+--------+------------------+
|    2006|18.655857950872626|
|    2007| 17.70617237497992|
|    2008|16.767700737100785|
|    2009|15.819573344509596|
+--------+------------------+



In [31]:
## Entrenamiento del modelo

In [32]:
## Nombres de las columnas de interés

In [33]:
##
## Se obtienen las columnas de las
## características de interes
##
inputCols = [a for a,_ in spark_df.dtypes]
inputCols = inputCols[3:-1]
inputCols

['friends',
 'basketball',
 'football',
 'soccer',
 'softball',
 'volleyball',
 'swimming',
 'cheerleading',
 'baseball',
 'tennis',
 'sports',
 'cute',
 'sex',
 'sexy',
 'hot',
 'kissed',
 'dance',
 'band',
 'marching',
 'music',
 'rock',
 'god',
 'church',
 'jesus',
 'bible',
 'hair',
 'dress',
 'blonde',
 'mall',
 'shopping',
 'clothes',
 'hollister',
 'abercrombie',
 'die',
 'death',
 'drunk',
 'drugs',
 'age1319']

In [34]:
## Ensamble de las columnas usadas en el clustering

In [35]:
## Se ensamblan las columnas en una lista por cada registro del DataFrame.

In [36]:
from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler(
    inputCols = inputCols,
    outputCol = 'rawFeatures')

spark_df = vectorAssembler.transform(spark_df)

spark_df.select('rawFeatures').show()

IllegalArgumentException: 'Output column rawFeatures already exists.'

In [ ]:
## Escalamiento de los datos
##   Se procede al escalamiento de los datos relevantes para el agrupamiento.

In [ ]:
from pyspark.ml.feature import MinMaxScaler

##
## Se construye el modelo y se entrena
##
scalerModel = MinMaxScaler(inputCol="rawFeatures", outputCol="features").fit(spark_df)

##
## Se aplica al DataFrame original para escalar los datos
##
spark_df = scalerModel.transform(spark_df)

##
## Se imprimen los datos escalados
##
spark_df.select(['features']).show()

In [29]:
## Determinación de los grupos

In [30]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

##
## Se realiza el agrupamiento. SetK() indica la
## cantidad de grupos para los que deben obtenerse
## sus centros.
##
model = KMeans().setK(5).setSeed(1).fit(spark_df)


spark_df = model.transform(spark_df)

silhouette = ClusteringEvaluator().evaluate(spark_df)


print("Silhouette with squared euclidean distance = " + str(silhouette))

Silhouette with squared euclidean distance = 0.4111397632582284


In [31]:
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Cluster Centers: 
[0.03319446 0.00894352 0.01386418 0.00704525 0.00718048 0.00752434
 0.00329701 0.00325638 0.00601355 0.00510883 0.00979218 0.01168149
 0.00109098 0.00634891 0.00791208 0.00176525 0.01047724 0.00405721
 0.00361525 0.00989082 0.00927761 0.00515989 0.00441119 0.00343609
 0.0017008  0.00643388 0.00628235 0.00016319 0.0098997  0.01184277
 0.         0.0031591  0.00252869 0.00657909 0.00662576 0.
 0.00188643]
[0.04732319 0.01824388 0.03681733 0.00983469 0.01404121 0.01513317
 0.00610534 0.32093534 0.01156427 0.00508475 0.01400659 0.03185813
 0.00263901 0.01129944 0.02429379 0.00727944 0.02231638 0.00395908
 0.00218285 0.01167461 0.01715093 0.00770579 0.00834617 0.00386064
 0.00243965 0.01786532 0.01647834 0.00088547 0.03495763 0.06471495
 0.02683616 0.02354049 0.01977401 0.00866718 0.0090799  0.01059322
 0.00512006]
[0.05157551 0.01875615 0.02536873 0.01379875 0.01931286 0.02498946
 0.00851651 0.00940675 0.00704277 0.0094002  0.01499508 0.04242871
 0.00224603 0.01052114 0.0

In [32]:
spark_df.select('prediction').show()

+----------+
|prediction|
+----------+
|         0|
|         2|
|         0|
|         0|
|         3|
|         3|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         4|
|         0|
|         0|
|         0|
|         0|
|         2|
+----------+
only showing top 20 rows



In [33]:
## Análisis del modelo

In [34]:
##
## Número de patrones asignados a cada cluster
##
spark_df.groupBy('prediction').count().toPandas()

,prediction,count
0,1,708
1,3,1592
2,4,2607
3,2,3361
4,0,21732


In [35]:
##
## clusters a los que pertenecen los primeros patrones
##
spark_df.select(["prediction", "gender", "age", "friends"]).show()

+----------+------+------+-------+
|prediction|gender|   age|friends|
+----------+------+------+-------+
|         0|     M|18.982|      7|
|         2|     F|18.801|      0|
|         0|     M|18.335|     69|
|         0|     F|18.875|      0|
|         3|    NA|18.995|     10|
|         3|     F|  null|    142|
|         0|     F| 18.93|     72|
|         0|     M|18.322|     17|
|         0|     F|19.055|     52|
|         0|     F|18.708|     39|
|         0|     F|18.543|      8|
|         0|     F|19.463|     21|
|         0|     F|18.097|     87|
|         0|    NA|  null|      0|
|         4|     F|18.398|      0|
|         0|    NA|  null|      0|
|         0|    NA|  null|    135|
|         0|     F|18.987|     26|
|         0|     F|17.158|     27|
|         2|     F|18.497|    123|
+----------+------+------+-------+
only showing top 20 rows



In [36]:
spark_df.join(age_df, spark_df.gradyear == age_df.gradyear)
spark_df.printSchema()

root
 |-- gradyear: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- friends: integer (nullable = true)
 |-- basketball: integer (nullable = true)
 |-- football: integer (nullable = true)
 |-- soccer: integer (nullable = true)
 |-- softball: integer (nullable = true)
 |-- volleyball: integer (nullable = true)
 |-- swimming: integer (nullable = true)
 |-- cheerleading: integer (nullable = true)
 |-- baseball: integer (nullable = true)
 |-- tennis: integer (nullable = true)
 |-- sports: integer (nullable = true)
 |-- cute: integer (nullable = true)
 |-- sex: integer (nullable = true)
 |-- sexy: integer (nullable = true)
 |-- hot: integer (nullable = true)
 |-- kissed: integer (nullable = true)
 |-- dance: integer (nullable = true)
 |-- band: integer (nullable = true)
 |-- marching: integer (nullable = true)
 |-- music: integer (nullable = true)
 |-- rock: integer (nullable = true)
 |-- god: integer (nullable = true)
 |-- church: inte

In [37]:
##
## Características demográficas de los clusters.
## Edad por cluster.
##
spark_df.groupby("prediction").mean().select(['prediction', 'avg(age1319)']).show()

+----------+------------------+
|prediction|      avg(age1319)|
+----------+------------------+
|         1|16.982795681063113|
|         3|17.393787091988127|
|         4|17.115304945054945|
|         2|17.101725897255488|
|         0|17.292400662819276|
+----------+------------------+



In [38]:
##
## Se agrega una columna indicadora para el genero
##
spark_df = spark_df.withColumn(
    'isFemale',
    when(spark_df['gender'] == 'F', 1).otherwise(0))

In [39]:
##
## Cantidad de mujeres por cluster.
##
spark_df.groupby("prediction").mean().select(['prediction', 'avg(isFemale)']).show()

+----------+------------------+
|prediction|     avg(isFemale)|
+----------+------------------+
|         1|0.9067796610169492|
|         3|0.7594221105527639|
|         4|0.8304564633678557|
|         2| 0.911335911930973|
|         0|0.6890760169335542|
+----------+------------------+



In [ ]:
spark.stop()